In [1]:
# importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels
import os 
import random 

In [2]:
import sklearn
import tensorflow
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score,mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [3]:
os.environ['PYTHONHASHSEED'] = '42'  

random.seed(42)          
np.random.seed(42)       
tensorflow.random.set_seed(42)   

In [4]:
# load the data into dataframe
df=pd.read_csv('feature_engineered_data.csv')
print(df.head())

   current_value  lights         T1       RH_1         T2       RH_2  \
0          430.0      30  20.133333  48.000000  19.566667  44.400000   
1          250.0      30  20.260000  52.726667  19.730000  45.100000   
2          100.0      10  20.426667  55.893333  19.856667  45.833333   
3          100.0      10  20.566667  53.893333  20.033333  46.756667   
4           90.0      10  20.730000  52.660000  20.166667  47.223333   

          T3       RH_3         T4       RH_4  ...  is_weekend    nsm  lag_1  \
0  19.890000  44.900000  19.000000  46.363333  ...           0  68400  576.6   
1  19.890000  45.493333  19.000000  47.223333  ...           0  69000  430.0   
2  20.033333  47.526667  19.000000  48.696667  ...           0  69600  250.0   
3  20.100000  48.466667  19.000000  48.490000  ...           0  70200  100.0   
4  20.200000  48.530000  18.926667  48.156667  ...           0  70800  100.0   

   lag_2  lag_6  lag_12  hour_sin  hour_cos  rolling_mean_12  rolling_std_12  
0  230.

In [5]:
def evaluate(model_name,target_scaler,y_test,y_pred):
   
    y_pred_actual = target_scaler.inverse_transform(y_pred)
    y_test_actual = target_scaler.inverse_transform(y_test.reshape(-1, 1))

    mae_GRU = mean_absolute_error(y_test_actual, y_pred_actual)
    r2_GRU = r2_score(y_test_actual, y_pred_actual)
    rmse_GRU = np.sqrt(mean_squared_error(y_test_actual, y_pred_actual))

    print(f"Model Name :{model_name}")
    print(f"MAE: {mae_GRU:.2f}")
    print(f"R² Score: {r2_GRU:.4f}")
    print(f"RMSE: {rmse_GRU:.4f}")

In [6]:
import pickle

# Load the selected features for 10-minute forecasting
with open("selected_features_60min.pkl", "rb") as f:
    selected_features = pickle.load(f)

print("Selected Features:", selected_features)

Selected Features: ['current_value', 'T3', 'RH_5', 'RH_8', 'T_out', 'Press_mm_hg', 'nsm', 'hour_cos', 'rolling_mean_12', 'rolling_std_12']


In [7]:
df['target_60min']= df['current_value'].shift(-6)

In [8]:
df = df.dropna().reset_index(drop=True)

In [9]:
df = df[selected_features + ['target_60min']]

In [10]:
train_size = int(0.8 * len(df))
df_train, df_test = df[:train_size], df[train_size:]

In [11]:
scaler = MinMaxScaler()
df_train_scaled = pd.DataFrame(scaler.fit_transform(df_train), columns=df.columns, index=df_train.index)
df_test_scaled = pd.DataFrame(scaler.transform(df_test), columns=df.columns, index=df_test.index)

In [12]:
def create_sequences(data, target_column, window_size):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data.iloc[i-window_size:i].values)
        y.append(data.iloc[i][target_column])
    return np.array(X), np.array(y)

window_size = 24  # use previous 24 time steps
X_train, y_train = create_sequences(df_train_scaled, target_column='target_60min', window_size=window_size)
X_test, y_test = create_sequences(df_test_scaled, target_column='target_60min', window_size=window_size)



In [13]:
target_scaler = MinMaxScaler()
target_scaler.fit(df[['target_60min']])

MinMaxScaler()

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv1D, MaxPooling1D, Flatten, LSTM, Dense, GRU,Dropout

model_cnn_lstm = Sequential([
    TimeDistributed(Conv1D(filters=64, kernel_size=2, activation='relu'), input_shape=(6, 4, 11)),
    TimeDistributed(MaxPooling1D(pool_size=2)),
    TimeDistributed(Flatten()),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)
])

model_cnn_lstm.compile(optimizer='adam', loss='mse')
model_cnn_lstm.summary()

# Reshape input for TimeDistributed
X_train_cnn = X_train.reshape((X_train.shape[0], 6,4,11))
X_test_cnn = X_test.reshape((X_test.shape[0], 6,4,11))

model_cnn_lstm.fit(X_train_cnn, y_train, epochs=10, batch_size=64, validation_split=0.1, verbose=1,shuffle=False)


c:\Users\Rukhsana\anaconda3\envs\project\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, 6, 3, 64)       │         1,472 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 6, 1, 64)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 6, 64)          │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,609 (143.00 KB)

 Trainable params: 36,609 (143.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0359 - val_loss: 0.0181
Epoch 2/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0288 - val_loss: 0.0174
Epoch 3/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0266 - val_loss: 0.0172
Epoch 4/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0262 - val_loss: 0.0168
Epoch 5/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0256 - val_loss: 0.0167
Epoch 6/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0252 - val_loss: 0.0165
Epoch 7/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0248 - val_loss: 0.0164
Epoch 8/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0246 - val_loss: 0.0161
Epoch 9/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0242 - val_loss: 0.0160
Epoch 10/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0240 - val_loss: 0.0161


In [15]:
y_pred_cnn = model_cnn_lstm.predict(X_test_cnn)

evaluate("CNN-LSTM",target_scaler,y_test,y_pred_cnn)


123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Model Name :CNN-LSTM
MAE: 35.98
R² Score: 0.3240
RMSE: 69.7262


In [16]:
model_cnn_gru = Sequential([
    TimeDistributed(Conv1D(filters=64, kernel_size=2, activation='relu'), input_shape=(6, 4, 11)),
    TimeDistributed(MaxPooling1D(pool_size=2)),
    TimeDistributed(Flatten()),
    GRU(64, return_sequences=False),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)
])

model_cnn_gru.compile(optimizer='adam', loss='mse')
model_cnn_gru.summary()

# Reshape input for TimeDistributed
X_train_cnn = X_train.reshape((X_train.shape[0], 6, 4, 11))
X_test_cnn = X_test.reshape((X_test.shape[0], 6,4,11))

model_cnn_gru.fit(X_train_cnn, y_train, epochs=10, batch_size=64, validation_split=0.1, verbose=1,shuffle=False)


c:\Users\Rukhsana\anaconda3\envs\project\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_3              │ (None, 6, 3, 64)       │         1,472 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 6, 1, 64)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 6, 64)          │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,545 (111.50 KB)

 Trainable params: 28,545 (111.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0353 - val_loss: 0.0175
Epoch 2/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0280 - val_loss: 0.0171
Epoch 3/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0269 - val_loss: 0.0171
Epoch 4/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0260 - val_loss: 0.0167
Epoch 5/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0250 - val_loss: 0.0166
Epoch 6/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0249 - val_loss: 0.0162
Epoch 7/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0246 - val_loss: 0.0164
Epoch 8/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0244 - val_loss: 0.0161
Epoch 9/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0240 - val_loss: 0.0162
Epoch 10/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0239 - val_loss: 0.0161


In [17]:
y_pred_cnn_gru = model_cnn_gru.predict(X_test_cnn)

evaluate("CNN-GRU",target_scaler,y_test,y_pred_cnn_gru)

123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Model Name :CNN-GRU
MAE: 37.09
R² Score: 0.3250
RMSE: 69.6785


In [18]:
#import keras_tcn
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU
from tcn import TCN  # keras-tcn package

model_tcn_lstm = Sequential([
    TCN(input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    LSTM(50),
    Dense(1)
])

model_tcn_lstm.compile(optimizer='adam', loss='mse')
model_tcn_lstm.summary()

model_tcn_lstm.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1, verbose=1,shuffle=False)


c:\Users\Rukhsana\anaconda3\envs\project\Lib\site-packages\tcn\tcn.py:268: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tcn (TCN)                       │ (None, 24, 64)         │       138,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        23,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 161,867 (632.29 KB)

 Trainable params: 161,867 (632.29 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 14s 48ms/step - loss: 0.0456 - val_loss: 0.0173
Epoch 2/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - loss: 0.0234 - val_loss: 0.0105
Epoch 3/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - loss: 0.0169 - val_loss: 0.0098
Epoch 4/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 46ms/step - loss: 0.0155 - val_loss: 0.0095
Epoch 5/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - loss: 0.0151 - val_loss: 0.0097
Epoch 6/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - loss: 0.0145 - val_loss: 0.0101
Epoch 7/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 47ms/step - loss: 0.0141 - val_loss: 0.0099
Epoch 8/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 11s 51ms/step - loss: 0.0138 - val_loss: 0.0108
Epoch 9/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 12s 52ms/step - loss: 0.0134 - val_loss: 0.0109
Epoch 10/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - loss: 0.0131 - val_loss: 0.0101


In [19]:
y_pred_tcn = model_tcn_lstm.predict(X_test)

evaluate("TCN-LSTM",target_scaler,y_test,y_pred_tcn)

123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step
Model Name :TCN-LSTM
MAE: 28.25
R² Score: 0.4875
RMSE: 60.7145


In [20]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU
from tcn import TCN  # keras-tcn package

model_tcn_gru = Sequential([
    TCN(input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    GRU(50),
    Dense(1)
])

model_tcn_gru.compile(optimizer='adam', loss='mse')
model_tcn_gru.summary()

model_tcn_gru.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1, verbose=1,shuffle=False)

c:\Users\Rukhsana\anaconda3\envs\project\Lib\site-packages\tcn\tcn.py:268: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tcn_1 (TCN)                     │ (None, 24, 64)         │       138,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 50)             │        17,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 156,267 (610.42 KB)

 Trainable params: 156,267 (610.42 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - loss: 0.1174 - val_loss: 0.0265
Epoch 2/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 46ms/step - loss: 0.0367 - val_loss: 0.0236
Epoch 3/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 45ms/step - loss: 0.0356 - val_loss: 0.0235
Epoch 4/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - loss: 0.0358 - val_loss: 0.0189
Epoch 5/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - loss: 0.0322 - val_loss: 0.0205
Epoch 6/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 45ms/step - loss: 0.0330 - val_loss: 0.0196
Epoch 7/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - loss: 0.0289 - val_loss: 0.0169
Epoch 8/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 45ms/step - loss: 0.0223 - val_loss: 0.0103
Epoch 9/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 46ms/step - loss: 0.0164 - val_loss: 0.0099
Epoch 10/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - loss: 0.0158 - val_loss: 0.0100


In [21]:
y_pred_tcn_gru = model_tcn_lstm.predict(X_test)
evaluate("TCN-GRU",target_scaler,y_test,y_pred_tcn_gru)

123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Model Name :TCN-GRU
MAE: 28.25
R² Score: 0.4875
RMSE: 60.7145
